In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
!pip install -q google-generativeai python-dotenv


In [6]:
!pip install -U google-generativeai


In [5]:
import google.generativeai as genai
import os
from dotenv import load_dotenv


In [8]:
GOOGLE_API_KEY="GOOGLE_API_KEY"


In [9]:
# Load Google API key
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Google API Key not found in Kaggle Secrets.")

print("✅ Google API Key loaded successfully.")

# Correct configuration
genai.configure(api_key=GOOGLE_API_KEY)

# Correct model initialization
model = genai.GenerativeModel("gemini-2.5-flash")

# Correct API call
response = model.generate_content("Hello! Can you summarize this?")

# Print output
print("✅ Gemini test successful!")
print(response.text)


✅ Google API Key loaded successfully.
✅ Gemini test successful!
Hello there!

Absolutely, I can summarize something for you.

However, it looks like you haven't provided the text yet.

Please paste or type the content you'd like me to summarize, and I'll be happy to help!


In [10]:
# Step 3: Test request
model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello! Can you summarize this?")

print(response.text)


Hello!

Yes, I can definitely summarize something for you!

However, it seems you forgot to provide the text, article, document, or whatever it is you'd like me to summarize.

Please paste or type the text here, provide a link, or upload the file, and I'll be happy to give you a concise summary!


In [11]:
# Step 4: Define system instructions and run a structured query

model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=(
        "You are an Enterprise Demo Agent. "
        "Your job is to answer user questions clearly and concisely."
    )
)

query = "Explain what an Enterprise AI Agent is in simple words."

response = model.generate_content(query)

print(response.text)


An Enterprise AI Agent is a smart software program that uses Artificial Intelligence to perform specific tasks or make decisions for a business.

Think of it as a specialized, automated assistant that helps companies streamline operations, answer questions, or handle routine work more efficiently.


In [12]:
# STEP 5 — Gemini Tool Calling (Complete Working Code for Kaggle)

import json
import google.generativeai as genai

# 1. Define the tool schema (JSON schema, not Python functions)
tools = [
    {
        "function_declarations": [
            {
                "name": "get_employee_details",
                "description": "Fetch role and experience of an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"}
                    },
                    "required": ["name"]
                }
            }
        ]
    }
]

# 2. Mock Python function — executed manually after Gemini calls the tool
def get_employee_details(name: str):
    employee_db = {
        "Unnati": {"role": "Frontend Developer", "experience": "2 years"},
        "Priyanka": {"role": "Team Lead", "experience": "5 years"},
        "Bhoomi": {"role": "Backend Developer", "experience": "3 years"}
    }
    return employee_db.get(name, {"error": "Employee not found"})


# 3. Build model with tool schema
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=tools
)

# 4. Trigger tool call
response = model.generate_content("Get details for employee Unnati")

print("RAW RESPONSE:\n", response)

# 5. Extract tool call arguments (Kaggle-safe version)
parts = response.candidates[0].content.parts

if hasattr(parts[0], "function_call") and parts[0].function_call:
    call = parts[0].function_call
    args = dict(call.args)  # convert MapComposite to normal dict

    print("\nFunction call detected:")
    print("Name:", call.name)
    print("Args:", args)

    # Execute our mock tool
    result = get_employee_details(args["name"])

    print("\nTool Execution Result:", result)
else:
    print("No tool call detected.")


RAW RESPONSE:
 response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_employee_details",
                  "args": {
                    "name": "Unnati"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 49,
        "candidates_token_count": 18,
        "total_token_count": 133
      },
      "model_version": "gemini-2.5-flash"
    }),
)

Function call detected:
Name: get_employee_details
Args: {'name': 'Unnati'}

Tool Execution Result: {'role': 'Frontend Developer', 'experience': '2 years'}


In [15]:
# STEP 6 — Full Automatic Agent Loop (FIXED)

import google.generativeai as genai
import json

# ---- 1. Tool Schema ----
tools = [
    {
        "function_declarations": [
            {
                "name": "get_employee_details",
                "description": "Fetch role and experience of an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"}
                    },
                    "required": ["name"]
                }
            }
        ]
    }
]

# ---- 2. Actual Python function ----
def get_employee_details(name: str):
    database = {
        "Unnati": {"role": "Frontend Developer", "experience": "2 years"},
        "Priyanka": {"role": "Team Lead", "experience": "5 years"},
        "Bhoomi": {"role": "Backend Developer", "experience": "3 years"},
    }
    return database.get(name, {"error": "Employee not found"})


# ---- 3. Initialize Model ----
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=tools
)


# ---- 4. Agent Loop ----
def agent_loop(user_input):
    print(f"\nUSER: {user_input}")

    # Phase 1 — User → Gemini
    response = model.generate_content(user_input)
    parts = response.candidates[0].content.parts

    # Phase 2 — Tool call?
    if hasattr(parts[0], "function_call") and parts[0].function_call:
        call = parts[0].function_call
        args = dict(call.args)

        print("\n🔧 Gemini requested tool:", call.name)
        print("Arguments:", args)

        # Run the Python tool
        result = get_employee_details(args["name"])
        print("\n🛠 Tool Result:", result)

        # Phase 3 — Return result back to Gemini
        final_response = model.generate_content(
            [
                {"role": "user", "parts": user_input},
                {
                    "role": "tool",
                    "parts": [
                        {
                            "function_response": {
                                "name": call.name,
                                "response": result   # FIXED: dict, not string
                            }
                        }
                    ]
                }
            ]
        )

        print("\n🤖 AGENT RESPONSE:")
        print(final_response.text)

    else:
        print("\n🤖 AGENT RESPONSE:")
        print(parts[0].text)


# ---- 5. Test it ----
agent_loop("Get details for employee Unnati")



USER: Get details for employee Unnati

🔧 Gemini requested tool: get_employee_details
Arguments: {'name': 'Unnati'}

🛠 Tool Result: {'role': 'Frontend Developer', 'experience': '2 years'}

🤖 AGENT RESPONSE:
Unnati is a Frontend Developer with 2 years of experience.


In [16]:
# STEP 7 — Add multiple tools + Universal agent

import google.generativeai as genai
import json

# ---- 1. Define All Tools ----
tools = [
    {
        "function_declarations": [
            {
                "name": "get_employee_details",
                "description": "Fetch role and experience of an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {"name": {"type": "string"}},
                    "required": ["name"]
                }
            },
            {
                "name": "get_salary_details",
                "description": "Get salary breakdown for an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {"name": {"type": "string"}},
                    "required": ["name"]
                }
            },
            {
                "name": "get_leave_balance",
                "description": "Fetch remaining leave days.",
                "parameters": {
                    "type": "object",
                    "properties": {"name": {"type": "string"}},
                    "required": ["name"]
                }
            },
            {
                "name": "get_product_info",
                "description": "Get price and stock of a product.",
                "parameters": {
                    "type": "object",
                    "properties": {"product": {"type": "string"}},
                    "required": ["product"]
                }
            }
        ]
    }
]

# ---- 2. Python Implementations ----

def get_employee_details(name: str):
    db = {
        "Unnati": {"role": "Frontend Developer", "experience": "2 years"},
        "Priyanka": {"role": "Team Lead", "experience": "5 years"},
        "Bhoomi": {"role": "Backend Developer", "experience": "3 years"}
    }
    return db.get(name, {"error": "Employee not found"})


def get_salary_details(name: str):
    salary = {
        "Unnati": {"basic": 35000, "hra": 15000, "bonus": 5000},
        "Priyanka": {"basic": 60000, "hra": 20000, "bonus": 10000}
    }
    return salary.get(name, {"error": "Salary record not found"})


def get_leave_balance(name: str):
    leaves = {
        "Unnati": {"remaining_leaves": 12},
        "Priyanka": {"remaining_leaves": 5},
        "Bhoomi": {"remaining_leaves": 9}
    }
    return leaves.get(name, {"error": "Leave info not found"})


def get_product_info(product: str):
    products = {
        "Laptop": {"price": 55000, "stock": 34},
        "Mouse": {"price": 799, "stock": 120},
        "Keyboard": {"price": 1499, "stock": 75}
    }
    return products.get(product, {"error": "Product not found"})


# ---- 3. Universal Tool Executor ----

def execute_tool(name, args):
    if name == "get_employee_details":
        return get_employee_details(**args)
    if name == "get_salary_details":
        return get_salary_details(**args)
    if name == "get_leave_balance":
        return get_leave_balance(**args)
    if name == "get_product_info":
        return get_product_info(**args)

    return {"error": f"Unknown tool {name}"}


# ---- 4. Initialize Model ----
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=tools
)


# ---- 5. Multi-Tool Agent Loop ----
def agent(query):
    print("\nUSER:", query)

    # Step A — Ask the model
    response = model.generate_content(query)
    part = response.candidates[0].content.parts[0]

    # Step B — If tool call is requested
    if hasattr(part, "function_call") and part.function_call:
        call = part.function_call
        args = dict(call.args)

        print("\n🔧 Tool needed:", call.name)
        print("Arguments:", args)

        # Step C — Execute appropriate Python function
        tool_result = execute_tool(call.name, args)
        print("🛠 Tool Output:", tool_result)

        # Step D — Send result back to Gemini
        final = model.generate_content(
            [
                {"role": "user", "parts": query},
                {
                    "role": "tool",
                    "parts": [
                        {
                            "function_response": {
                                "name": call.name,
                                "response": tool_result
                            }
                        }
                    ]
                }
            ]
        )

        print("\n🤖 AGENT:")
        print(final.text)
        return final.text

    else:
        # No tool needed → Normal text response
        print("\n🤖 AGENT:")
        print(part.text)
        return part.text


# ---- 6. Test ----
agent("What is the salary of Priyanka?")



USER: What is the salary of Priyanka?

🔧 Tool needed: get_salary_details
Arguments: {'name': 'Priyanka'}
🛠 Tool Output: {'basic': 60000, 'hra': 20000, 'bonus': 10000}

🤖 AGENT:
Priyanka's salary is 90K (60K Basic + 10K Bonus + 20K HRA).


"Priyanka's salary is 90K (60K Basic + 10K Bonus + 20K HRA)."

In [17]:
# STEP 7 — Add multiple enterprise functions / tools

# ---- 1. Define more tools ----
tools = [
    {
        "function_declarations": [
            {
                "name": "get_employee_details",
                "description": "Fetch role and experience of an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"}
                    },
                    "required": ["name"]
                }
            },
            {
                "name": "get_employee_salary",
                "description": "Return salary of an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"}
                    },
                    "required": ["name"]
                }
            },
            {
                "name": "list_all_employees",
                "description": "Returns a list of all employees in the company.",
                "parameters": {
                    "type": "object",
                    "properties": {}
                }
            }
        ]
    }
]


# ---- 2. Define actual Python functions ----
def get_employee_details(name: str):
    employees = {
        "Unnati": {"role": "Frontend Developer", "experience": "2 years"},
        "Priyanka": {"role": "Team Lead", "experience": "5 years"},
        "Bhoomi": {"role": "Backend Developer", "experience": "3 years"},
    }
    return employees.get(name, {"error": "Employee not found"})


def get_employee_salary(name: str):
    salaries = {
        "Unnati": 65000,
        "Priyanka": 90000,
        "Bhoomi": 70000,
    }
    return {"salary": salaries.get(name, "Not available")}


def list_all_employees():
    return ["Unnati", "Priyanka", "Bhoomi"]


# ---- 3. Reinitialize the agent with new tools ----
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=tools
)

print("✅ Tools updated successfully!")


✅ Tools updated successfully!


In [19]:
# STEP 8 — Universal Agent Loop (supports all tools)

import json

# Map tool names → Python functions
tool_router = {
    "get_employee_details": get_employee_details,
    "get_employee_salary": get_employee_salary,
    "list_all_employees": lambda: list_all_employees()
}


def agent(query):
    print(f"\n🔵 USER: {query}")

    # Step 1 — Ask Gemini
    response = model.generate_content(query)
    part = response.candidates[0].content.parts[0]

    # Step 2 — Check if Gemini wants to call a tool
    if hasattr(part, "function_call") and part.function_call:
        call = part.function_call
        tool_name = call.name
        tool_args = dict(call.args)

        print(f"\n🧰 Gemini requested tool: {tool_name}")
        print("Arguments:", tool_args)

        # Step 3 — Run actual Python function
        if tool_name in tool_router:
            if tool_args:
                result = tool_router[tool_name](**tool_args)
            else:
                result = tool_router[tool_name]()  # no args tool
        else:
            result = {"error": "Unknown tool"}

        print("\n🛠 Tool Output:", result)

        # Step 4 — Send result back to Gemini
        final = model.generate_content(
            [
                {"role": "user", "parts": query},
                {
                    "role": "tool",
                    "parts": [
                        {
                            "function_response": {
                                "name": tool_name,
                                "response": result
                            }
                        }
                    ]
                }
            ]
        )

        print("\n🤖 AGENT RESPONSE:")
        print(final.text)

    else:
        # If no tool call — direct response
        print("\n🤖 AGENT RESPONSE:")
        print(part.text)


In [21]:
# STEP 9 — Full Conversational Agent

chat_history = []  # stores all messages in this session

def conversational_agent(user_input):
    print(f"\n🔵 USER: {user_input}")
    
    # Add user input to history
    chat_history.append({"role": "user", "parts": user_input})
    
    # Ask Gemini with full history
    response = model.generate_content(chat_history)
    part = response.candidates[0].content.parts[0]

    # Check for tool call
    if hasattr(part, "function_call") and part.function_call:
        call = part.function_call
        tool_name = call.name
        tool_args = dict(call.args)

        print(f"\n🧰 Gemini requested tool: {tool_name}")
        print("Arguments:", tool_args)

        # Run Python tool
        if tool_name in tool_router:
            if tool_args:
                result = tool_router[tool_name](**tool_args)
            else:
                result = tool_router[tool_name]()
        else:
            result = {"error": "Unknown tool"}

        print("\n🛠 Tool Output:", result)

        # Add tool result to history
        chat_history.append({
            "role": "tool",
            "parts": [{"function_response": {"name": tool_name, "response": result}}]
        })

        # Send back to Gemini to generate final response
        final = model.generate_content(chat_history)
        chat_history.append({"role": "assistant", "parts": final.text})

        print("\n🤖 AGENT RESPONSE:")
        print(final.text)

    else:
        # Direct response from Gemini
        chat_history.append({"role": "assistant", "parts": part.text})
        print("\n🤖 AGENT RESPONSE:")
        print(part.text)


In [39]:
import google.generativeai as genai
import json

# ---- 0. Initialize Model and Tools ----
chat_history = []

tools = [
    {
        "function_declarations": [
            {
                "name": "get_employee_details",
                "description": "Fetch role, experience, and salary of an employee.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"}
                    },
                    "required": ["name"]
                }
            }
        ]
    }
]

# Employee database function
def get_employee_details(name: str):
    database = {
        "Unnati": {"role": "Frontend Developer", "experience": "2 years", "salary": "₹6 LPA"},
        "Priyanka": {"role": "Team Lead", "experience": "5 years", "salary": "₹12 LPA"},
        "Bhoomi": {"role": "Backend Developer", "experience": "3 years", "salary": "₹8 LPA"},
    }
    return database.get(name, {"error": "Employee not found"})

tool_router = {
    "get_employee_details": get_employee_details
}

# Initialize Gemini model
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=tools
)

# ---- Helper to extract employee name and requested field ----
def extract_employee_name_and_field(user_input):
    user_input_lower = user_input.lower()
    name = None
    field = "details"
    
    for emp in ["Unnati", "Priyanka", "Bhoomi"]:
        if emp.lower() in user_input_lower:
            name = emp
            break
    
    if "salary" in user_input_lower:
        field = "salary"
    elif "experience" in user_input_lower:
        field = "experience"
    
    return name, field

# ---- 1. Robust Agent with Fallback and Unknown Handling ----
def robust_agent(user_input):
    print(f"\n🔵 USER: {user_input}")
    chat_history.append({"role": "user", "parts": [user_input]})

    try:
        name, field = extract_employee_name_and_field(user_input)
        if name:
            result = get_employee_details(name)
            chat_history.append({
                "role": "tool",
                "parts": [{"function_response": {"name": "get_employee_details", "response": result}}]
            })

            if "error" in result:
                final_text = f"⚠️ {result['error']}"
            else:
                if field == "details":
                    final_text = f"{name} is a {result['role']} with {result['experience']} of experience, and an annual salary of {result['salary']}."
                else:
                    final_text = f"{name}'s {field} is {result.get(field, 'not found')}."
            
            chat_history.append({"role": "assistant", "parts": [final_text]})
            print("\n🤖 AGENT RESPONSE:")
            print(final_text)
        else:
            fallback_msg = "⚠️ Sorry, employee not recognized or no valid information found."
            chat_history.append({"role": "assistant", "parts": [fallback_msg]})
            print("\n🤖 AGENT RESPONSE:")
            print(fallback_msg)

    except Exception as e:
        fallback_msg = f"⚠️ Sorry, I couldn't process your request due to: {str(e)}"
        chat_history.append({"role": "assistant", "parts": [fallback_msg]})
        print("\n🤖 AGENT RESPONSE:")
        print(fallback_msg)


# ---- 2. Test the updated agent ----
robust_agent("Get details for employee Unnati")
robust_agent("What is the salary of Priyanka?")
robust_agent("Tell me the experience of Bhoomi")
robust_agent("Get details for employee Unknown")



🔵 USER: Get details for employee Unnati

🤖 AGENT RESPONSE:
Unnati is a Frontend Developer with 2 years of experience, and an annual salary of ₹6 LPA.

🔵 USER: What is the salary of Priyanka?

🤖 AGENT RESPONSE:
Priyanka's salary is ₹12 LPA.

🔵 USER: Tell me the experience of Bhoomi

🤖 AGENT RESPONSE:
Bhoomi's experience is 3 years.

🔵 USER: Get details for employee Unknown

🤖 AGENT RESPONSE:
⚠️ Sorry, employee not recognized or no valid information found.
